<a href="https://colab.research.google.com/github/merucode/DL/blob/01-colab-study_must_have_pytorch/06-%5Bregression-rnn%5D-stock-price-regression-basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## STEP 0. Version check and Install Dependency

Step 0-1. Version Check

In [ ]:
import sys
import torch
print(f"Python version:{sys.version}")                  # python
print("Torch version:{}".format(torch.__version__))     # torch
print("cuda version: {}".format(torch.version.cuda))    # cuda
print("cudnn version:{}".format(torch.backends.cudnn.version()))    # cudnn

Step 0-2. Install Dependency

In [ ]:
!pip install pykrx

## STEP 1. Check Data

Step 1-1. Load data

In [ ]:
import pandas as pd
from pykrx import stock

df = stock.get_market_ohlcv("20200720", "20230716", "005930") # 삼성전자 ohclv 불러오기
df.head()

Step 1-2. Check data type

In [ ]:
import matplotlib.pyplot as plt

data_used = df.iloc[:, 0:3]
data_used.columns = ['Open', 'High', 'Low']
data_used['Close'] = df['종가']
hist = data_used.hist()
plt.show()

## STEP 2. Dataset

Step 2-1. Custom Dataset

In [ ]:
import numpy as np

from pykrx import stock

from torch.utils.data.dataset import Dataset

class Samsung(Dataset):  # ❶ 클래스 선언
    def __init__(self):
        self.df = df = stock.get_market_ohlcv("20200720", "20230716", "005930")   # 삼성전자 ohclv 불러오기

        # 입력 데이터 정규화
        self.data = self.df.iloc[:, 0:3].values   # ❸ 종가를 제외한 데이터
        self.data = self.data / np.max(self.data)  # ➍ 0과 1 사이로 정규화

        # ➎ 종가 데이터 정규화
        self.label = self.df["종가"].values
        self.label = self.label / np.max(self.label)

    # 사용 가능한 배치 개수 반환
    def __len__(self):
        return len(self.data) - 30 # ❶ 사용 가능한 배치 개수
        # 이론적으로 N-L+1이나 마지막 종가는 정답으로 사용하기 때문에 N-L

    # 1개의 배치 아이템
    def __getitem__(self, i):
        data = self.data[i:i+30] # ❶ 입력 데이터 30일치 읽기
        label = self.label[i+30] # ❷ 종가 데이터 30일치 읽기

        return data, label

Step 2-2. DataLoader

In [ ]:
from torch.utils.data.dataloader import DataLoader

dataset = Samsung()  # 데이터셋의 정의
loader = DataLoader(dataset, batch_size=32)  # 배치 크기를 32로 설정

## STEP 3. Module

In [ ]:
import torch
import torch.nn as nn

class RNN(nn.Module):
   def __init__(self):
       super(RNN, self).__init__()

       self.rnn = nn.RNN(input_size=3, hidden_size=8, num_layers=5,
                         batch_first=True)  # ❶ RNN층의 정의

       # ❷ 주가를 예측하는 MLP층 정의
       self.fc1 = nn.Linear(in_features=240, out_features=64)
       self.fc2 = nn.Linear(in_features=64, out_features=1)

       self.relu = nn.ReLU() # 활성화 함수 정의

   def forward(self, x, h0):
       x, hn = self.rnn(x, h0)  # ❶ RNN층의 출력

       # ❷ MLP층의 입력으로 사용될 수 있도록 모양 변경
       x = torch.reshape(x, (x.shape[0], -1))

       # MLP 층을 이용해 종가를 예측
       x = self.fc1(x)
       x = self.relu(x)
       x = self.fc2(x)

       # 예측한 종가를 1차원 벡터로 표현
       x = torch.flatten(x)

       return x

## STEP 4. Learning

Step 4-1. Setting

In [ ]:
import tqdm

from torch.optim.adam import Adam

device = "cuda" if torch.cuda.is_available() else "cpu" # # 학습을 진행할 프로세서 설정

model = RNN().to(device) # 모델의 정의

lr = 1e-4 # 학습률 정의

optim = Adam(params=model.parameters(), lr=lr) # 사용할 최적화를 설정

Step 4-2. Learning

In [ ]:
for epoch in range(200):
   iterator = tqdm.tqdm(loader)
   for data, label in iterator:
       optim.zero_grad()

       # ❶ 초기 은닉 상태(shape: 은닉층 개수, 배치 크기, 출력 차원)
       h0 = torch.zeros(5, data.shape[0], 8).to(device)

       # ❷ 모델의 예측값
       pred = model(data.type(torch.FloatTensor).to(device), h0)

       # ❸ 손실의 계산
       loss = nn.MSELoss()(pred, label.type(torch.FloatTensor).to(device))
       loss.backward()  # 오차 역전파
       optim.step()  # 최적화 진행

       iterator.set_description(f"epoch{epoch} loss:{loss.item()}")

torch.save(model.state_dict(), "./rnn.pth")  # 모델 저장

## STEP 5. Evaluation

Step 5-1. Evaluation

In [ ]:
import matplotlib.pyplot as plt

loader = DataLoader(dataset, batch_size=1)  # 예측값을 위한 데이터 로더

preds = []  # 예측값들을 저장하는 리스트
total_loss = 0

with torch.no_grad():
   # 모델의 가중치 불러오기
   model.load_state_dict(torch.load("rnn.pth", map_location=device))

   for data, label in loader:
       h0 = torch.zeros(5, data.shape[0], 8).to(device)  # ➊초기 은닉상태 정의

       # 모델의 예측값 출력
       pred = model(data.type(torch.FloatTensor).to(device), h0)
       preds.append(pred.item())  # ➋예측값을 리스트에 추가
       loss = nn.MSELoss()(pred, label.type(torch.FloatTensor).to(device))  # 손실계산
       total_loss += loss/len(loader)  # ➌손실의 평균치 계산

total_loss.item()

Step 5-2. Plot

In [ ]:
plt.plot(preds, label="prediction")
plt.plot(dataset.label[30:], label="actual")
plt.legend()
plt.show()